## Google Colab

In [25]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

# Change working path
import os
path = "/content/drive/MyDrive/Colab Notebooks/DeepLearning/Lab03"
os.chdir(path)
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/DeepLearning/Lab03


# Task description
- Identity the speakers of given features.
- Main goal: Get familiar with transformer.

In [26]:
# unzip the file
# !tar zxvf Dataset.tar.gz

## Fix Random Seed

In [27]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)                        # 为numpy设置随机种子
    random.seed(seed)                           # 为python设置随机种子
    torch.manual_seed(seed)                     # 为CPU设置随机种子
    if torch.cuda.is_available():               # 如果有可用的GPU
        torch.cuda.manual_seed(seed)            # 为当前GPU设置随机种子
        torch.cuda.manual_seed_all(seed)        # 为所有GPU设置随机种子
    torch.backends.cudnn.benchmark = False      # 避免使用CUDNN算法
    torch.backends.cudnn.deterministic = True   # 避免使用CUDNN算法

set_seed(87)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training.
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary.
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [28]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


class myDataset(Dataset):
	def __init__(self, data_dir, segment_len=128):
		self.data_dir = data_dir
		self.segment_len = segment_len

		# Load the mapping from speaker neme to their corresponding id.
		mapping_path = Path(data_dir) / "mapping.json"
		mapping = json.load(mapping_path.open())	# json file to python dict
		self.speaker2id = mapping["speaker2id"]

		# Load metadata of training data.
		metadata_path = Path(data_dir) / "metadata.json"
		metadata = json.load(open(metadata_path))["speakers"]

		# Get the total number of speaker.
		self.speaker_num = len(metadata.keys())
		self.data = []	# data = [[file_0, id_0], [file_1, id_1], ...]
		for speaker in metadata.keys():
			for utterances in metadata[speaker]:
				self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

	def __len__(self):
			return len(self.data)

	def __getitem__(self, index):
		feat_path, speaker = self.data[index]
		# Load preprocessed mel-spectrogram.
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		# Segmemt mel-spectrogram into "segment_len" frames.
		if len(mel) > self.segment_len:
			# Randomly get the starting point of the segment.
			start = random.randint(0, len(mel) - self.segment_len)
			# Get a segment with "segment_len" frames.
			mel = torch.FloatTensor(mel[start:start+self.segment_len])
		else:
			mel = torch.FloatTensor(mel)
		# Turn the speaker id into long for computing loss later.
		speaker = torch.FloatTensor([speaker]).long()
		return mel, speaker

	def get_speaker_number(self):
		return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [29]:
import torch
from torch.utils.data import DataLoader, random_split	# random_split is used to split dataset
from torch.nn.utils.rnn import pad_sequence				# pad_sequence is used to pad


def collate_batch(batch):
	# Process features within a batch.
	"""Collate a batch of data."""
	mel, speaker = zip(*batch)
	# Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
	mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
	# mel: (batch size, length, 40)
	return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
	"""Generate dataloader"""
	dataset = myDataset(data_dir)
	speaker_num = dataset.get_speaker_number()
	# Split dataset into training dataset and validation dataset
	trainlen = int(0.9 * len(dataset))
	lengths = [trainlen, len(dataset) - trainlen]
	trainset, validset = random_split(dataset, lengths)

	train_loader = DataLoader(
		trainset,
		batch_size=batch_size,
		shuffle=True,
		drop_last=True,
		num_workers=n_workers,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	valid_loader = DataLoader(
		validset,
		batch_size=batch_size,
		num_workers=n_workers,
		drop_last=True,
		pin_memory=True,
		collate_fn=collate_batch,
	)

	return train_loader, valid_loader, speaker_num

# Model
- Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
    - TransformerEncoderLayer:
    - TransformerEncoder:

## Sublayer

In [30]:
import torch.nn as nn
import torch.nn.functional as F

'''Attention function'''
def attention(query, key, value, dropout=None):
    # Scaled Dot-Product Attention
    # Attention(Q,K,V) = softmax( Q(K).T / sqrt(d_k) )V
    d_k = query.size(-1) # last dimanison
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    p_attn = F.softmax(scores, dim=-1)

    if dropout is not None:
        p_attn = dropout(p_attn)

    return torch.matmul(p_attn, value), p_attn

'''Multi-Head Attention Network'''
class MultiHeadAttention(nn.Module):
    def __init__(self, nhead, d_model, dropout=0.1):
        super(MultiHeadAttention, self).__init__()

        self.d_k = d_model // nhead
        self.nhead = nhead
        # Projection Matrix
        self.wq = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.wk = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.wv = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.wo = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        # Attention Matrix
        self.attn = None
        # Dropout Layer
        self.dropout = nn.Dropout(p=dropout)


    def forward(self, query, key, value):
        # Batch size
        nbatches = query.size(0)

        # query, key, value = [matrix(x).view(nbatches, -1, self.nhead, self.d_k).transpose(1, 2)
        #     for matrix, x in zip((self.wq, self.wk, self.wv), (query, key, value))]
        query = self.wq(query).view(nbatches, -1, self.nhead, self.d_k).transpose(1, 2)
        key = self.wk(key).view(nbatches, -1, self.nhead, self.d_k).transpose(1, 2)
        value = self.wv(value).view(nbatches, -1, self.nhead, self.d_k).transpose(1, 2)

        x, self.attn = attention(query, key, value, dropout=self.dropout)
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.nhead * self.d_k)
        x = self.wo(x)

        return x

'''Position-wise Feed-Forward Network'''
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(FeedForward, self).__init__()
        self.hidden1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.hidden2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.hidden2(self.dropout(F.relu(self.hidden1(x))))

'''Add & Norm'''
class AddandNorm(nn.Module):
    def __init__(self, size):
        super(AddandNorm, self).__init__()
        self.norm = nn.LayerNorm(size)

    def forward(self, x, layer):
        return self.norm(x + layer(x))


## Decoder

In [31]:
import copy

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, dim_feedforward, nhead, dropout):
        super().__init__()
        self.attention = MultiHeadAttention(nhead, d_model, dropout)
        self.res1 = AddandNorm(d_model)
        self.ff = FeedForward(d_model, dim_feedforward, dropout)
        self.res2 = AddandNorm(d_model)

    def forward(self, x):
        out = self.res1(x, lambda x: self.attention(x, x, x))
        out = self.res2(out, self.ff)
        return out

class TransformerEncoder(nn.Module):
    def __init__(self, num_layers, encoder_layer):
        super().__init__()
        self.encoders = nn.ModuleList([copy.deepcopy(encoder_layer) for i in range(num_layers)])

    def forward(self, x):
        for layer in self.encoders:
            x = layer(x)

        return x

## Classifier

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
	def __init__(self, N=3, d_model= 128, n_spks=600, d_ff=256, nhead=8, dropout=0.1):
		super().__init__()
		# Project the dimension of features from that of input into d_model.
		self.prenet = nn.Linear(40, d_model)
		# TODO:
		#   Build vanilla transformer encoder block from scratch !!!
        #   You can't call the transformer encoder block function from PyTorch library !!!
        #   The number of encoder layers should be less than 3 !!!
        #   The parameter size of transformer encoder block should be less than 500k !!!
		# self.encoder_layer = #your own transformer encoder layer
		self.encoder = TransformerEncoder(N, TransformerEncoderLayer(d_model, d_ff, nhead, dropout))
		# Project the the dimension of features from d_model into speaker nums.
		self.pred_layer = nn.Sequential(
			nn.Linear(d_model, d_model),
			nn.ReLU(),
			nn.Linear(d_model, n_spks),
		)

	def forward(self, mels):
		out = self.prenet(mels)
		out = self.encoder(out)
		# mean pooling
		stats = out.mean(dim=1)
		out = self.pred_layer(stats)
		return out

encoderblock = Classifier()
param_enc = sum(p.numel() for p in encoderblock.encoder.parameters())
print (f"The parameter size of encoder block is {param_enc/1000}k")

The parameter size of encoder block is 395.904k


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [33]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
):
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [34]:
import torch


def model_fn(batch, model, criterion, device):
	"""Forward a batch through the model."""

	mels, labels = batch
	mels = mels.to(device)
	labels = labels.to(device)

	outs = model(mels)

	loss = criterion(outs, labels)

	# Get the speaker id with highest probability.
	preds = outs.argmax(1)
	# Compute accuracy.
	accuracy = torch.mean((preds == labels).float())

	return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [35]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device):
	"""Validate on validation set."""

	model.eval()
	running_loss = 0.0
	running_accuracy = 0.0
	pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

	for i, batch in enumerate(dataloader):
		with torch.no_grad():
			loss, accuracy = model_fn(batch, model, criterion, device)
			running_loss += loss.item()
			running_accuracy += accuracy.item()

		pbar.update(dataloader.batch_size)
		pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)

	pbar.close()
	model.train()

	return running_accuracy / len(dataloader)

# Main function

In [36]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"save_path": "model.ckpt",
		"batch_size": 128,
		"n_workers": 16,
		"valid_steps": 2000,
		"warmup_steps": 1000,
		"save_steps": 10000,
		"total_steps": 70000,
	}

	return config


def main(
	data_dir,
	save_path,
	batch_size,
	n_workers,
	valid_steps,
	warmup_steps,
	total_steps,
	save_steps,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
	train_iterator = iter(train_loader)
	print(f"[Info]: Finish loading data!",flush = True)

	model = Classifier(n_spks=speaker_num).to(device)
	criterion = nn.CrossEntropyLoss()
	optimizer = AdamW(model.parameters(), lr=1e-3)
	scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
	print(f"[Info]: Finish creating model!",flush = True)

	best_accuracy = -1.0
	best_state_dict = None

	pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

	for step in range(total_steps):
		# Get data
		try:
			batch = next(train_iterator)
		except StopIteration:
			train_iterator = iter(train_loader)
			batch = next(train_iterator)

		loss, accuracy = model_fn(batch, model, criterion, device)
		batch_loss = loss.item()
		batch_accuracy = accuracy.item()

		# Updata model
		loss.backward()
		optimizer.step()
		scheduler.step()
		optimizer.zero_grad()

		# Log
		pbar.update()
		pbar.set_postfix(
			loss=f"{batch_loss:.2f}",
			accuracy=f"{batch_accuracy:.2f}",
			step=step + 1,
		)

		# Do validation
		if (step + 1) % valid_steps == 0:
			pbar.close()

			valid_accuracy = valid(valid_loader, model, criterion, device)

			# keep the best model
			if valid_accuracy > best_accuracy:
				best_accuracy = valid_accuracy
				best_state_dict = model.state_dict()

			pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

		# Save the best model so far.
		if (step + 1) % save_steps == 0 and best_state_dict is not None:
			torch.save(best_state_dict, save_path)
			pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

	pbar.close()


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[Info]: Finish loading data!
[Info]: Finish creating model!


Train:  20% 398/2000 [12:06<41:33,  1.56s/ step, accuracy=0.05, loss=5.19, step=398]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Train: 100% 2000/2000 [13:39<00:00,  2.44 step/s, accuracy=0.44, loss=2.29, step=2000]
Valid:  99% 5632/5667 [01:26<00:00, 64.95 uttr/s, accuracy=0.43, loss=2.56]
Train: 100% 2000/2000 [01:54<00:00, 17.42 step/s, accuracy=0.69, loss=1.41, step=4000]
Valid:  99% 5632/5667 [00:03<00:00, 1561.45 uttr/s, accuracy=0.58, loss=1.78]
Train: 100% 2000/2000 [01:55<00:00, 17.28 step/s, accuracy=0.70, loss=1.12, step=6000]
Valid:  99% 5632/5667 [00

Step 10000, best model saved. (accuracy=0.6998)


...

Step 50000, best model saved. (accuracy=0.8466)


Train: 100% 2000/2000 [01:51<00:00, 17.95 step/s, accuracy=0.98, loss=0.06, step=52000]
Valid:  99% 5632/5667 [00:03<00:00, 1565.98 uttr/s, accuracy=0.85, loss=0.83]
Train: 100% 2000/2000 [01:53<00:00, 17.55 step/s, accuracy=0.98, loss=0.06, step=54000]
Valid:  99% 5632/5667 [00:04<00:00, 1347.20 uttr/s, accuracy=0.85, loss=0.84]
Train: 100% 2000/2000 [01:50<00:00, 18.04 step/s, accuracy=0.98, loss=0.04, step=56000]
Valid:  99% 5632/5667 [00:03<00:00, 1497.92 uttr/s, accuracy=0.85, loss=0.80]
Train: 100% 2000/2000 [01:50<00:00, 18.04 step/s, accuracy=0.98, loss=0.08, step=58000]
Valid:  99% 5632/5667 [00:03<00:00, 1580.58 uttr/s, accuracy=0.86, loss=0.78]
Train: 100% 2000/2000 [01:51<00:00, 17.94 step/s, accuracy=0.99, loss=0.02, step=6e+4]
Valid:  99% 5632/5667 [00:03<00:00, 1422.78 uttr/s, accuracy=0.85, loss=0.79]
Train:   1% 12/2000 [00:00<00:34, 58.18 step/s, accuracy=1.00, loss=0.02, step=6e+4]

Step 60000, best model saved. (accuracy=0.8585)


Train: 100% 2000/2000 [01:50<00:00, 18.10 step/s, accuracy=0.98, loss=0.08, step=62000]
Valid:  99% 5632/5667 [00:03<00:00, 1537.25 uttr/s, accuracy=0.86, loss=0.79]
Train: 100% 2000/2000 [01:49<00:00, 18.23 step/s, accuracy=0.99, loss=0.02, step=64000]
Valid:  99% 5632/5667 [00:03<00:00, 1449.96 uttr/s, accuracy=0.86, loss=0.74]
Train: 100% 2000/2000 [01:50<00:00, 18.15 step/s, accuracy=0.99, loss=0.01, step=66000]
Valid:  99% 5632/5667 [00:03<00:00, 1514.05 uttr/s, accuracy=0.86, loss=0.77]
Train: 100% 2000/2000 [01:50<00:00, 18.13 step/s, accuracy=0.99, loss=0.03, step=68000]
Valid:  99% 5632/5667 [00:03<00:00, 1683.02 uttr/s, accuracy=0.86, loss=0.79]
Train: 100% 2000/2000 [01:50<00:00, 18.15 step/s, accuracy=0.99, loss=0.05, step=7e+4]
Valid:  99% 5632/5667 [00:03<00:00, 1438.43 uttr/s, accuracy=0.86, loss=0.78]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 70000, best model saved. (accuracy=0.8612)


# Inference

## Dataset of inference

In [37]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel


def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [38]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torchsummary import summary
from torch.utils.data import DataLoader

def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"model_path": "./model.ckpt",
		"output_path": "./output.csv",
	}

	return config


def main(
	data_dir,
	model_path,
	output_path,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
		num_workers=8,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model = Classifier(n_spks=speaker_num).to(device)
	model.load_state_dict(torch.load(model_path))
	model.eval()
	print(f"[Info]: Finish creating model!",flush = True)

	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
		with torch.no_grad():
			mels = mels.to(device)
			outs = model(mels)
			preds = outs.argmax(1).cpu().numpy()
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/8000 [00:00<?, ?it/s]